In [ ]:
%%capture
!pip install neptune-client psutil
!git clone https://github.com/Cho-D-YoungRae/URP_PD.git
%cd URP_PD
!pwd

In [ ]:
import dataset
import object_detection
from utils import *

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.transforms.functional as FT

import os
import json
from PIL import Image
import numpy as np
import argparse
from tqdm.auto import tqdm
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using <{device}> device")

Using <cuda> device


In [ ]:
from torch.backends import cudnn
cudnn.benchmark = True

# ====== Random Seed Initialization ====== #
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.baselineID = 2

# ====== Dataset ====== #
args.img_type = 'lwir'
args.val_split = 0.1

# ====== Model ====== #
args.base_model = 'VGGBase'
args.n_classes = len(label_map)
args.one_ch_option = "mean"


# ====== Optimizer & Training ====== #
args.optim = 'Adam' 
args.lr = 1e-3
args.twice_b_lr = True
args.weight_decay = 5e-4

args.decay_lr_at = [80000, 100000]
args.decay_lr_to = 0.1

args.iteration = 120000
args.train_batch_size = 8
args.test_batch_size = 16

In [ ]:
checkpoint = os.path.join('/content/drive/MyDrive/2021.summer_URP/PD/checkpoint',
                          str(args.baselineID)+'.pth.tar')
checkpoint = checkpoint if os.path.isfile(checkpoint) else None
print(f"checkpoint: {checkpoint}")

checkpoint: /content/drive/MyDrive/2021.summer_URP/PD/checkpoint/2.pth.tar


In [ ]:
import neptune.new as neptune

api_token = 
run = neptune.init(project='jodyr/urp',
                   api_token=api_token)

run["parameters"] = vars(args)


KeyboardInterrupt: ignored

-----
## code 작성

In [ ]:
def train(train_loader, model, criterion, optimizer):
    """
    One epoch's training.

    :param train_loader: DataLoader for training data
    :param model: model
    :param criterion: MultiBox loss
    :param optimizer: optimizer
    :param epoch: epoch number
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.train()  # training mode enables dropout

    batch_time = AverageMeter()  # forward prop. + back prop. time
    data_time = AverageMeter()  # data loading time
    losses = AverageMeter()  # loss

    print_freq = 200

    start = time.time()

    # Batches
    for i, (images, bboxes, category_ids, is_crowds) in enumerate(train_loader):
        data_time.update(time.time() - start)

        images = images.to(device)  # (batch_size (N), 3, 300, 300)
        bboxes = [b.to(device) for b in bboxes]
        category_ids = [c.to(device) for c in category_ids]

        # Forward prop.
        predicted_locs, predicted_scores = model(images)  # (N, 8732, 4), (N, 8732, n_classes)

        # Loss
        loss = criterion(predicted_locs, predicted_scores, bboxes, category_ids)  # scalar

        # Backward prop.
        optimizer.zero_grad()
        loss.backward()

        # Update model
        optimizer.step()

        losses.update(loss.item(), images.size(0))
        batch_time.update(time.time() - start)

        start = time.time()

        # Print status
        if i % print_freq == 0:
            print(f'[{i}/{len(train_loader)}]\t'
                  f'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  f'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  f'Loss {losses.val:.4f} ({losses.avg:.4f})\t')
    del predicted_locs, predicted_scores, images, bboxes, category_ids

    train_loss = losses.avg
    return train_loss        

In [ ]:
def validation(val_loader, model, criterion):
    model.eval()

    num_batches = len(val_loader)
    losses = AverageMeter()
    with torch.no_grad():
        for i, (images, boxes, labels, _) in enumerate(val_loader):
            images = images.to(device)
            boxes = [b.to(device) for b in boxes]
            labels = [l.to(device) for l in labels]

            predicted_locs, predicted_scores = model(images)
            loss = criterion(predicted_locs, predicted_scores, boxes, labels).item()

            losses.update(loss, images.size(0))

    val_loss = losses.avg
    return val_loss

In [ ]:
if checkpoint is None:
    start_epoch = 0
    lr = args.lr
    model = object_detection.SSD300(n_classes=args.n_classes,
                                    base=args.base_model,
                                    one_ch_option=args.one_ch_option)
    if args.twice_b_lr:
        biases = list()
        not_biases = list()
        for param_name, param in model.named_parameters():
            if param.requires_grad:
                if param_name.endswith('.bias'):
                    biases.append(param)
                else:
                    not_biases.append(param)
        optimizer = getattr(torch.optim, args.optim)(params=[{'params': biases, 'lr': 2 * lr}, 
                                                            {'params': not_biases}],
                                                     lr=lr,
                                                     weight_decay=args.weight_decay)
    else:
        optimizer = getattr(torch.optim, args.optim)(params=model.parameters(),
                                                     lr=lr,
                                                     weight_decay=args.weight_decay)

else:
    checkpoint = torch.load(checkpoint)
    start_epoch = checkpoint['epoch'] + 1
    print('\nLoaded checkpoint from epoch %d.\n' % checkpoint['epoch'])
    model = checkpoint['model']
    optimizer = checkpoint['optimizer']


model = model.to(device)
criterion = object_detection.MultiBoxLoss(priors_cxcy=model.priors_cxcy).to(device)


Loaded checkpoint from epoch 25.



/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [ ]:
train_dataset = dataset.KaistPDDataset()

total_train_size = len(train_dataset)
val_size = int(total_train_size * args.val_split)
train_val_split = [total_train_size - val_size, val_size]
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, 
                                                           train_val_split)

In [ ]:
workers = 4
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=args.train_batch_size, 
                                           shuffle=True,
                                           collate_fn=dataset.collate_fn,
                                           num_workers=workers,
                                           pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_dataset, 
                                         batch_size=args.test_batch_size, 
                                         shuffle=False,
                                         collate_fn=dataset.collate_fn,
                                         num_workers=workers,
                                         pin_memory=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
checkpoint_dir = '/content/drive/MyDrive/2021.summer_URP/PD/checkpoint'
checkpoint_path = os.path.join(checkpoint_dir,
                               str(args.baselineID)+'.pth.tar')

epochs = args.iteration // (len(train_dataset) // args.train_batch_size)
decay_lr_at = [it // (len(train_dataset) // args.train_batch_size) for it in args.decay_lr_at]
val_save_freq = 5
train_losses = []
val_losses = []


# Epochs
for epoch in range(start_epoch, epochs):
    print(f"# ====== Epoch {epoch} ====== #")
    # Decay learning rate at particular epochs
    if epoch in decay_lr_at:
        adjust_learning_rate(optimizer, args.decay_lr_to)

    # One epoch's training
    train_loss = train(train_loader=train_loader,
                        model=model,
                        criterion=criterion,
                        optimizer=optimizer)

    # validation & save
    if epoch % val_save_freq == 0:
        val_loss = validation(val_loader, model, criterion)
        print(f"train loss: {train_loss}, val loss: {val_loss}")
        run['train/loss'].log(train_loss)
        run['val/loss'].log(val_loss)
        if val_losses and val_losses[-1] >= val_loss:
            print(f"save checkpoint at [Epoch {epoch}]")
            save_checkpoint(epoch, model, optimizer, checkpoint_path)
        train_losses.append(train_loss)
        val_losses.append(val_loss)

# ====== Epoch 0 ====== #


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[0/983]	Batch Time 79.609 (79.609)	Data Time 76.182 (76.182)	Loss 17.1773 (17.1773)	
[200/983]	Batch Time 4.939 (1.792)	Data Time 4.760 (1.594)	Loss 3.3519 (4.0067)	
[400/983]	Batch Time 5.059 (1.587)	Data Time 4.880 (1.397)	Loss 3.3237 (3.7400)	
[600/983]	Batch Time 4.777 (1.516)	Data Time 4.588 (1.329)	Loss 3.1814 (3.5839)	
[800/983]	Batch Time 5.039 (1.480)	Data Time 4.859 (1.295)	Loss 3.1238 (3.4515)	
train loss: 3.3497605618438877, val loss: 2.753069313940723
# ====== Epoch 1 ====== #
[0/983]	Batch Time 0.654 (0.654)	Data Time 0.423 (0.423)	Loss 2.6791 (2.6791)	
[200/983]	Batch Time 0.184 (0.188)	Data Time 0.000 (0.002)	Loss 2.8173 (2.7345)	
[400/983]	Batch Time 0.188 (0.186)	Data Time 0.000 (0.001)	Loss 2.8565 (2.7062)	
[600/983]	Batch Time 0.184 (0.186)	Data Time 0.000 (0.001)	Loss 2.8287 (2.6472)	
[800/983]	Batch Time 0.186 (0.185)	Data Time 0.000 (0.001)	Loss 2.5235 (2.6156)	
# ====== Epoch 2 ====== #
[0/983]	Batch Time 0.752 (0.752)	Data Time 0.547 (0.547)	Loss 3.0201 (3.0201

Error occurred during asynchronous operation processing: Cannot log infinite or NaN value to attribute train/loss
Error occurred during asynchronous operation processing: Cannot log infinite or NaN value to attribute val/loss


[200/983]	Batch Time 0.185 (0.188)	Data Time 0.000 (0.002)	Loss nan (nan)	
[400/983]	Batch Time 0.183 (0.186)	Data Time 0.000 (0.001)	Loss nan (nan)	
[600/983]	Batch Time 0.185 (0.186)	Data Time 0.000 (0.001)	Loss nan (nan)	
[800/983]	Batch Time 0.186 (0.185)	Data Time 0.000 (0.001)	Loss nan (nan)	
# ====== Epoch 37 ====== #
[0/983]	Batch Time 0.758 (0.758)	Data Time 0.550 (0.550)	Loss nan (nan)	
[200/983]	Batch Time 0.186 (0.188)	Data Time 0.000 (0.003)	Loss nan (nan)	
[400/983]	Batch Time 0.183 (0.186)	Data Time 0.000 (0.002)	Loss nan (nan)	
[600/983]	Batch Time 0.185 (0.186)	Data Time 0.000 (0.001)	Loss nan (nan)	
[800/983]	Batch Time 0.185 (0.186)	Data Time 0.000 (0.001)	Loss nan (nan)	
# ====== Epoch 38 ====== #
[0/983]	Batch Time 0.767 (0.767)	Data Time 0.553 (0.553)	Loss nan (nan)	
[200/983]	Batch Time 0.186 (0.188)	Data Time 0.000 (0.003)	Loss nan (nan)	
[400/983]	Batch Time 0.185 (0.186)	Data Time 0.000 (0.002)	Loss nan (nan)	
[600/983]	Batch Time 0.186 (0.186)	Data Time 0.000

...

Error occurred during asynchronous operation processing: Cannot log infinite or NaN value to attribute train/loss
Error occurred during asynchronous operation processing: Cannot log infinite or NaN value to attribute val/loss


[200/983]	Batch Time 0.184 (0.189)	Data Time 0.000 (0.003)	Loss nan (nan)	
[400/983]	Batch Time 0.186 (0.187)	Data Time 0.000 (0.002)	Loss nan (nan)	
[600/983]	Batch Time 0.186 (0.187)	Data Time 0.000 (0.001)	Loss nan (nan)	
[800/983]	Batch Time 0.185 (0.186)	Data Time 0.000 (0.001)	Loss nan (nan)	
# ====== Epoch 72 ====== #
[0/983]	Batch Time 0.583 (0.583)	Data Time 0.353 (0.353)	Loss nan (nan)	
[200/983]	Batch Time 0.186 (0.188)	Data Time 0.000 (0.002)	Loss nan (nan)	
[400/983]	Batch Time 0.184 (0.187)	Data Time 0.000 (0.001)	Loss nan (nan)	
[600/983]	Batch Time 0.187 (0.186)	Data Time 0.002 (0.001)	Loss nan (nan)	
[800/983]	Batch Time 0.185 (0.186)	Data Time 0.000 (0.001)	Loss nan (nan)	
# ====== Epoch 73 ====== #
[0/983]	Batch Time 0.781 (0.781)	Data Time 0.542 (0.542)	Loss nan (nan)	
[200/983]	Batch Time 0.185 (0.189)	Data Time 0.000 (0.003)	Loss nan (nan)	
[400/983]	Batch Time 0.184 (0.187)	Data Time 0.000 (0.002)	Loss nan (nan)	
[600/983]	Batch Time 0.186 (0.187)	Data Time 0.000

Error occurred during asynchronous operation processing: Cannot log infinite or NaN value to attribute train/loss
Error occurred during asynchronous operation processing: Cannot log infinite or NaN value to attribute val/loss


[200/983]	Batch Time 0.186 (0.189)	Data Time 0.000 (0.003)	Loss nan (nan)	
[400/983]	Batch Time 0.186 (0.187)	Data Time 0.000 (0.002)	Loss nan (nan)	
[600/983]	Batch Time 0.186 (0.186)	Data Time 0.000 (0.001)	Loss nan (nan)	
[800/983]	Batch Time 0.184 (0.186)	Data Time 0.000 (0.001)	Loss nan (nan)	
# ====== Epoch 77 ====== #
[0/983]	Batch Time 0.679 (0.679)	Data Time 0.427 (0.427)	Loss nan (nan)	
[200/983]	Batch Time 0.183 (0.188)	Data Time 0.000 (0.002)	Loss nan (nan)	
[400/983]	Batch Time 0.185 (0.186)	Data Time 0.000 (0.001)	Loss nan (nan)	
[600/983]	Batch Time 0.184 (0.186)	Data Time 0.000 (0.001)	Loss nan (nan)	
[800/983]	Batch Time 0.186 (0.186)	Data Time 0.000 (0.001)	Loss nan (nan)	
# ====== Epoch 78 ====== #
[0/983]	Batch Time 0.834 (0.834)	Data Time 0.589 (0.589)	Loss nan (nan)	
[200/983]	Batch Time 0.184 (0.188)	Data Time 0.000 (0.003)	Loss nan (nan)	
[400/983]	Batch Time 0.185 (0.187)	Data Time 0.000 (0.002)	Loss nan (nan)	
[600/983]	Batch Time 0.184 (0.186)	Data Time 0.001

NameError: ignored

------


In [6]:
import eval

eval.evaluate(model,
              min_score=0.2,
              max_overlap=0.5,
              top_k=200)

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/content/URP_PD/object_detection.py:393: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_boxes.append(class_decoded_locs[1 - suppress])
/content/URP_PD/object_detection.py:395: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_scores.append(class_scores[1 - suppress])



Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.36s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Miss Rate  (MR) @ Reasonable         [ IoU=0.50      | height=[55:10000000000] | visibility=[none+partial_occ] ] = 43.63%
Recall: 0.6974729241877257
